### Problem sentence


You have to create two separate functions that are not allowed to comunicate between them i.e. can't use global variables.
I generate an array of size "array_size" with random integers with values less than "max_int" (values can repeat).
This array is passed to the first function that is tasked to arrange the elements in the array in whatever manner it sees fit and returns the newly ordered array to me.
I take a random element out of the array and I pass it to the second function.
The second function is tasked with finding what element I removed from the array.

Please fill in the two functions with your code.

Good luck! :)


### Import Libraries


In [51]:
import numpy as np


In [52]:
max_int: int = 10000
array_size: int = 100000

### Generate array of random numbers


In [53]:
array: np.ndarray = np.random.randint(0, max_int, size=array_size)

### Run through first program


In [54]:
def function_1(bag):
    S = np.sum(bag)
    permutation = sorted(bag, key=lambda x: x ^ S)
    return permutation

In [55]:
array_1 = function_1(array)


### Remove a number from the list


In [56]:
np.random.seed()
idx = np.random.randint(0, len(array_1))
removed_element = array_1[idx]
array_1_modified = np.delete(array_1, idx)

In [57]:
removed_element

np.int32(3303)

### Run through second program


In [58]:
import numpy as np
from joblib import Parallel, delayed
import numba


@numba.njit(fastmath=True)
def is_sorted_ascending(arr):
    for i in range(len(arr) - 1):
        if arr[i] > arr[i + 1]:
            return False
    return True


@numba.njit(fastmath=True)
def find_g_in_range(truncated_perm, S_trunc, g_start, g_end):
    n = len(truncated_perm)
    keys = np.empty(n, dtype=np.uint64)
    for g in range(g_start, g_end):
        S_candidate = S_trunc + g

        for i in range(n):
            keys[i] = truncated_perm[i] ^ S_candidate

        if not is_sorted_ascending(keys):
            continue

        g_xor = g ^ S_candidate

        lo, hi = 0, n
        while lo < hi:
            mid = (lo + hi) >> 1
            if keys[mid] < g_xor:
                lo = mid + 1
            else:
                hi = mid
        pos = lo

        if pos > 0 and keys[pos - 1] > g_xor:
            continue
        if pos < n and g_xor > keys[pos]:
            continue

        return g
    return -1


def _process_chunk(truncated_perm, S_trunc, start_g, end_g):
    return find_g_in_range(truncated_perm, S_trunc, start_g, end_g)


def function_2(truncated_perm, n_jobs=-1, chunk_size=100):
    truncated_perm = np.ascontiguousarray(truncated_perm.ravel(), dtype=np.uint64)
    S_trunc = truncated_perm.sum(dtype=np.uint64)

    n_chunks = (max_int + chunk_size - 1) // chunk_size

    results = Parallel(n_jobs=n_jobs)(
        delayed(_process_chunk)(
            truncated_perm, S_trunc, i * chunk_size, min((i + 1) * chunk_size, max_int)
        )
        for i in range(n_chunks)
    )

    valid_g = [r for r in results if r != -1]
    return min(valid_g) if valid_g else None


In [59]:
answer = function_2(array_1_modified)

In [60]:
answer

3303